In [24]:
# ! pip3 install -U textblob --user
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
# from wordcloud import STOPWORDS,WordCloud
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pandas as pd
from pprint import pprint
import string
import os
import re
import pandas as pd
from textblob import TextBlob
pd.__version__


'1.4.2'

In [27]:
# read data from the sources
data= pd.read_json('/Users/apple/Desktop/10 academy/data/global_twitter_data.json',lines=True)


In [95]:
user= data['user']
user[0]['screen_name']

'i_ameztoy'

In [96]:
data['user'][0]

{'id': 3418339671,
 'id_str': '3418339671',
 'name': 'Iban Ameztoy',
 'screen_name': 'i_ameztoy',
 'location': '',
 'description': 'It´s not all about fancy words, but place always matters. Earth Observer 🛰🌍 Now @EU_scienceHub All Views My Own 🗺',
 'url': None,
 'entities': {'description': {'urls': []}},
 'protected': False,
 'followers_count': 20497,
 'friends_count': 2621,
 'listed_count': 232,
 'created_at': 'Wed Aug 12 17:31:24 +0000 2015',
 'favourites_count': 15760,
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': True,
 'verified': False,
 'statuses_count': 8097,
 'lang': None,
 'contributors_enabled': False,
 'is_translator': False,
 'is_translation_enabled': False,
 'profile_background_color': '000000',
 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png',
 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png',
 'profile_background_tile': False,
 'profile_image_url': 'http://pbs.twimg.com/profile_images

In [55]:
# display few raws
df=data.copy()

In [56]:
df.head(5)

,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,...,favorite_count,favorited,retweeted,lang,possibly_sensitive,extended_entities,quoted_status_id,quoted_status_id_str,quoted_status,withheld_in_countries
0,2022-08-07 22:31:20+00:00,1556407673284861952,1556407673284861952,RT @i_ameztoy: Extra random image (I):\n\nLets...,False,"[0, 143]","{'hashtags': [{'text': 'City', 'indices': [132...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-08-07 22:31:16+00:00,1556407654373027840,1556407654373027840,RT @IndoPac_Info: #China's media explains the ...,False,"[0, 140]","{'hashtags': [{'text': 'China', 'indices': [18...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-08-07 22:31:07+00:00,1556407616208773122,1556407616208773120,"China even cut off communication, they don't a...",False,"[0, 161]","{'hashtags': [{'text': 'XiJinping', 'indices':...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",1.556408e+18,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-08-07 22:31:06+00:00,1556407614602461185,1556407614602461184,"Putin to #XiJinping : I told you my friend, Ta...",False,"[0, 179]","{'hashtags': [{'text': 'XiJinping', 'indices':...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-08-07 22:31:04+00:00,1556407604238245889,1556407604238245888,"RT @ChinaUncensored: I’m sorry, I thought Taiw...",False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
# check null values
df.isnull().sum()

created_at                       0
id                               0
id_str                           0
full_text                        0
truncated                        0
display_text_range               0
entities                         0
metadata                         0
source                           0
in_reply_to_status_id        21103
in_reply_to_status_id_str    21103
in_reply_to_user_id          21032
in_reply_to_user_id_str      21032
in_reply_to_screen_name      21032
user                             0
geo                          22000
coordinates                  22000
place                        21883
contributors                 22000
retweeted_status              4740
is_quote_status                  0
retweet_count                    0
favorite_count                   0
favorited                        0
retweeted                        0
lang                             0
possibly_sensitive           15809
extended_entities            18641
quoted_status_id    

In [58]:
# drop null values
df.dropna()


,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,...,favorite_count,favorited,retweeted,lang,possibly_sensitive,extended_entities,quoted_status_id,quoted_status_id_str,quoted_status,withheld_in_countries


In [60]:
len(df)


22000

In [62]:
df.columns

Index(['created_at', 'id', 'id_str', 'full_text', 'truncated',
       'display_text_range', 'entities', 'metadata', 'source',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'retweeted_status', 'is_quote_status', 'retweet_count',
       'favorite_count', 'favorited', 'retweeted', 'lang',
       'possibly_sensitive', 'extended_entities', 'quoted_status_id',
       'quoted_status_id_str', 'quoted_status', 'withheld_in_countries'],
      dtype='object')

In [64]:
# some statistics of the dataframe
df.describe()

,id,id_str,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,geo,coordinates,contributors,retweet_count,favorite_count,possibly_sensitive,quoted_status_id,quoted_status_id_str
count,2.200000e+04,2.200000e+04,8.970000e+02,8.970000e+02,9.680000e+02,9.680000e+02,0.0,0.0,0.0,22000.000000,22000.000000,6191.000000,1.393000e+03,1.393000e+03
mean,1.556202e+18,1.556202e+18,1.555061e+18,1.555061e+18,6.303641e+17,6.303641e+17,NaN,NaN,NaN,177.295636,2.140682,0.037151,1.551431e+18,1.551431e+18
std,1.166818e+14,1.166818e+14,2.114087e+16,2.114087e+16,6.408578e+17,6.408578e+17,NaN,NaN,NaN,498.320518,42.884353,0.189146,4.656781e+16,4.656781e+16
min,1.555978e+18,1.555978e+18,9.329339e+17,9.329339e+17,4.283330e+05,4.283330e+05,NaN,NaN,NaN,0.000000,0.000000,0.000000,4.622719e+17,4.622719e+17
25%,1.556111e+18,1.556111e+18,1.555937e+18,1.555937e+18,1.416272e+08,1.416272e+08,NaN,NaN,NaN,3.000000,0.000000,0.000000,1.555738e+18,1.555738e+18
50%,1.556216e+18,1.556216e+18,1.556124e+18,1.556124e+18,7.574441e+17,7.574441e+17,NaN,NaN,NaN,39.000000,0.000000,0.000000,1.555930e+18,1.555930e+18
75%,1.556297e+18,1.556297e+18,1.556264e+18,1.556264e+18,1.266422e+18,1.266422e+18,NaN,NaN,NaN,187.000000,0.000000,0.000000,1.556156e+18,1.556156e+18
max,1.556408e+18,1.556408e+18,1.556408e+18,1.556408e+18,1.555982e+18,1.555982e+18,NaN,NaN,NaN,17409.000000,3267.000000,1.000000,1.556400e+18,1.556400e+18


In [73]:
# df['clean_text'] not in the columns

In [65]:
# data types
df.dtypes

created_at                   datetime64[ns, UTC]
id                                         int64
id_str                                     int64
full_text                                 object
truncated                                   bool
display_text_range                        object
entities                                  object
metadata                                  object
source                                    object
in_reply_to_status_id                    float64
in_reply_to_status_id_str                float64
in_reply_to_user_id                      float64
in_reply_to_user_id_str                  float64
in_reply_to_screen_name                   object
user                                      object
geo                                      float64
coordinates                              float64
place                                     object
contributors                             float64
retweeted_status                          object
is_quote_status     

In [76]:
#  engineer clean_text from full_text
text= df['full_text'][0]
print(text)
print()
print('##############################')
re.sub(r'[^\w]', ' ', text)

RT @i_ameztoy: Extra random image (I):

Lets focus in one very specific zone of the western coast -&gt; Longjing District, Taichung #City, #Ta…

##############################


'RT  i_ameztoy  Extra random image  I    Lets focus in one very specific zone of the western coast   gt  Longjing District  Taichung  City   Ta '

In [80]:
#  engineer clean_text from full_text
df['clean_text']=df['full_text'].apply(lambda text: re.sub(r'[^\w]', ' ', text))
df['clean_text']

0        RT  i_ameztoy  Extra random image  I    Lets f...
1        RT  IndoPac_Info   China s media explains the ...
2        China even cut off communication  they don t a...
3        Putin to  XiJinping   I told you my friend  Ta...
4        RT  ChinaUncensored  I m sorry  I thought Taiw...
                               ...                        
21995    RT  IndoPac_Info  A good infographic of  China...
21996    RT  IndoPac_Info  A good infographic of  China...
21997                   Reuters Thanks  Pelosi smart move 
21998    RT  IndoPac_Info   Taiwan people s Desire for ...
21999    RT  IndoPac_Info   Taiwan people s Desire for ...
Name: clean_text, Length: 22000, dtype: object

In [86]:
columns= df.columns
columns=['created_at', 'id', 'id_str', 'full_text','clean_text', 'truncated',
       'display_text_range', 'entities', 'metadata', 'source',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'retweeted_status', 'is_quote_status', 'retweet_count',
       'favorite_count', 'favorited', 'retweeted', 'lang',
       'possibly_sensitive', 'extended_entities', 'quoted_status_id',
       'quoted_status_id_str', 'quoted_status', 'withheld_in_countries'
       ]
# sort the columns
tweets_df =df[columns]

tweets_df

,created_at,id,id_str,full_text,clean_text,truncated,display_text_range,entities,metadata,source,...,favorite_count,favorited,retweeted,lang,possibly_sensitive,extended_entities,quoted_status_id,quoted_status_id_str,quoted_status,withheld_in_countries
0,2022-08-07 22:31:20+00:00,1556407673284861952,1556407673284861952,RT @i_ameztoy: Extra random image (I):\n\nLets...,RT i_ameztoy Extra random image I Lets f...,False,"[0, 143]","{'hashtags': [{'text': 'City', 'indices': [132...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-08-07 22:31:16+00:00,1556407654373027840,1556407654373027840,RT @IndoPac_Info: #China's media explains the ...,RT IndoPac_Info China s media explains the ...,False,"[0, 140]","{'hashtags': [{'text': 'China', 'indices': [18...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-08-07 22:31:07+00:00,1556407616208773122,1556407616208773120,"China even cut off communication, they don't a...",China even cut off communication they don t a...,False,"[0, 161]","{'hashtags': [{'text': 'XiJinping', 'indices':...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-08-07 22:31:06+00:00,1556407614602461185,1556407614602461184,"Putin to #XiJinping : I told you my friend, Ta...",Putin to XiJinping I told you my friend Ta...,False,"[0, 179]","{'hashtags': [{'text': 'XiJinping', 'indices':...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-08-07 22:31:04+00:00,1556407604238245889,1556407604238245888,"RT @ChinaUncensored: I’m sorry, I thought Taiw...",RT ChinaUncensored I m sorry I thought Taiw...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21995,2022-08-06 18:03:29+00:00,1555977877383938048,1555977877383938048,RT @IndoPac_Info: A good infographic of #China...,RT IndoPac_Info A good infographic of China...,False,"[0, 135]","{'hashtags': [{'text': 'China', 'indices': [40...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",...,0,False,False,en,0.0,"{'media': [{'id': 1555920926847295488, 'id_str...",NaN,NaN,NaN,NaN
21996,2022-08-06 18:03:27+00:00,1555977869746118661,1555977869746118656,RT @IndoPac_Info: A good infographic of #China...,RT IndoPac_Info A good infographic of China...,False,"[0, 135]","{'hashtags': [{'text': 'China', 'indices': [40...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",...,0,False,False,en,0.0,"{'media': [{'id': 1555920926847295488, 'id_str...",NaN,NaN,NaN,NaN
21997,2022-08-06 18:03:27+00:00,1555977869293223937,1555977869293223936,@Reuters Thanks #Pelosi smart move.,Reuters Thanks Pelosi smart move,False,"[9, 35]","{'hashtags': [{'text': 'Pelosi', 'indices': [1...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
21998,2022-08-06 18:03:26+00:00,1555977867174895616,1555977867174895616,RT @IndoPac_Info: #Taiwan people's Desire for ...,RT IndoPac_Info Taiwan people s Desire for ...,False,"[0, 140]","{'hashtags': [{'text': 'Taiwan', 'indices': [1...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
class PrepareData:
    def __init__(self,df):
        self.df=df
    
    def preprocess_data(self):
        tweets_df = self.df.loc[self.df['lang'] =="en"]
        #text Preprocessing
        tweets_df['clean_text']=tweets_df['clean_text'].astype(str)
        tweets_df['clean_text'] = tweets_df['clean_text'].apply(lambda x: x.lower())
        tweets_df['clean_text']= tweets_df['clean_text'].apply(lambda x: x.translate(
                                    str.maketrans(' ', ' ', string.punctuation)))
        
        #Converting tweets to list of words For feature engineering
        sentence_list = [tweet for tweet in tweets_df['clean_text']]
        word_list = [sent.split() for sent in sentence_list]
        # print(word_list)

        #Create dictionary which contains Id and word 
        word_to_id = corpora.Dictionary(word_list) #generate unique tokens
        #  we can see the word to unique integer mapping
        # print(word_to_id.token2id)
        # using bag of words(bow), we create a corpus that contains the word id
        #  and its frequency in each document.
        corpus_1= [word_to_id.doc2bow(tweet) for tweet in word_list]


        return word_list, word_to_id, corpus_1


In [89]:
PrepareData_obj=PrepareData(tweets_df)
word_list ,id2word,corpus=PrepareData_obj.preprocess_data()

In [104]:
id2word

In [90]:
word_list

[['rt',
  'iameztoy',
  'extra',
  'random',
  'image',
  'i',
  'lets',
  'focus',
  'in',
  'one',
  'very',
  'specific',
  'zone',
  'of',
  'the',
  'western',
  'coast',
  'gt',
  'longjing',
  'district',
  'taichung',
  'city',
  'ta'],
 ['rt',
  'indopacinfo',
  'china',
  's',
  'media',
  'explains',
  'the',
  'military',
  'reasons',
  'for',
  'each',
  'area',
  'of',
  'the',
  'drills',
  'in',
  'the',
  'taiwan',
  'strait',
  'read',
  'the',
  'labels',
  'in',
  'the',
  'pi'],
 ['china',
  'even',
  'cut',
  'off',
  'communication',
  'they',
  'don',
  't',
  'anwer',
  'phonecalls',
  'from',
  'the',
  'us',
  'but',
  'here',
  'clown',
  'zelenskyyua',
  'enters',
  'the',
  'stage',
  'to',
  'ask',
  'xijinping',
  'to',
  'change',
  'putin',
  's',
  'mind'],
 ['putin',
  'to',
  'xijinping',
  'i',
  'told',
  'you',
  'my',
  'friend',
  'taiwan',
  'will',
  'be',
  'a',
  'vassal',
  'state',
  'including',
  'nukes',
  'much',
  'like',
  'the',
  

In [98]:
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1)]


In [105]:
# get word's id
id_words=[ [(id2word[id],count) for id, count in line] for line in corpus]

In [108]:
print(id_words[0])

[('city', 1), ('coast', 1), ('district', 1), ('extra', 1), ('focus', 1), ('gt', 1), ('i', 1), ('iameztoy', 1), ('image', 1), ('in', 1), ('lets', 1), ('longjing', 1), ('of', 1), ('one', 1), ('random', 1), ('rt', 1), ('specific', 1), ('ta', 1), ('taichung', 1), ('the', 1), ('very', 1), ('western', 1), ('zone', 1)]


### Topic Modeling using Latent Dirichlet Allocation 
*  The purpose of LDA is mapping each teweets in our corpus to a set of topics 
which covers a good deal of the words in the tweet

In [114]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [115]:
pprint(lda_model.print_topics())

[(0,
  '0.037*"rt" + 0.036*"military" + 0.036*"on" + 0.029*"taiwan" + 0.028*"in" + '
  '0.028*"the" + 0.027*"chinese" + 0.022*"of" + 0.019*"drills" + 0.016*"s"'),
 (1,
  '0.053*"time" + 0.049*"what" + 0.040*"support" + 0.025*"can" + '
  '0.021*"respond" + 0.019*"c" + 0.019*"small" + 0.018*"into" + 0.017*"some" + '
  '0.016*"well"'),
 (2,
  '0.155*"an" + 0.043*"missiles" + 0.037*"country" + 0.035*"independent" + '
  '0.035*"uk" + 0.034*"if" + 0.027*"america" + 0.018*"want" + 0.017*"europe" + '
  '0.015*"unitedstates"'),
 (3,
  '0.055*"us" + 0.048*"to" + 0.047*"the" + 0.038*"pelosi" + 0.036*"rt" + '
  '0.030*"about" + 0.029*"visit" + 0.026*"by" + 0.025*"is" + 0.024*"and"'),
 (4,
  '0.028*"not" + 0.024*"this" + 0.023*"rt" + 0.021*"you" + 0.017*"taiwan" + '
  '0.016*"travel" + 0.016*"dead" + 0.016*"ccp" + 0.015*"your" + 0.015*"hotel"'),
 (5,
  '0.091*"the" + 0.073*"china" + 0.065*"taiwan" + 0.044*"rt" + 0.042*"s" + '
  '0.036*"is" + 0.033*"and" + 0.031*"in" + 0.024*"that" + 0.023*"a"'),
 (

In [116]:
pprint(lda_model.show_topics(formatted=False))

[(0,
  [('rt', 0.03738074),
   ('military', 0.035919663),
   ('on', 0.0357204),
   ('taiwan', 0.028654857),
   ('in', 0.028413307),
   ('the', 0.02792786),
   ('chinese', 0.027447347),
   ('of', 0.02156691),
   ('drills', 0.019273393),
   ('s', 0.016375583)]),
 (1,
  [('time', 0.05301942),
   ('what', 0.048676297),
   ('support', 0.039859835),
   ('can', 0.02461093),
   ('respond', 0.020784872),
   ('c', 0.019085603),
   ('small', 0.018663118),
   ('into', 0.017649896),
   ('some', 0.016793797),
   ('well', 0.015991282)]),
 (2,
  [('an', 0.15494052),
   ('missiles', 0.04271952),
   ('country', 0.03672991),
   ('independent', 0.03478951),
   ('uk', 0.03450579),
   ('if', 0.033801995),
   ('america', 0.027062254),
   ('want', 0.0176282),
   ('europe', 0.017225105),
   ('unitedstates', 0.014605355)]),
 (3,
  [('us', 0.054688074),
   ('to', 0.047953617),
   ('the', 0.047457583),
   ('pelosi', 0.038469758),
   ('rt', 0.035535518),
   ('about', 0.02998987),
   ('visit', 0.029286336),
   ('by

Each line is a topic with individual topic terms and weights. Topic0 can be termed as climate change, and Topic4 can be termed as government and carbon emission.

# **Model Analysis**

Perplexity is also a measure of model quality and in natural language processing is often used as “perplexity per number of words”. It describes how well a model predicts a sample, i.e. how much it is “perplexed” by a sample from the observed data. The lower the score, the better the model for the given data.

A coherence matrix is used to test the model for accuracy. Topic coherence is a measure that compares different topic models based on their human-interpretability. The coherence score ‘C_V’ provides a numerical value to the

In [117]:
# Compute Perplexity

#It's a measure of how good the model is. The lower the better. 
# Perplexity is a negative value
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
doc_lda = lda_model[corpus]


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, 
                                    texts=word_list, dictionary=id2word,
                                     coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n Ldamodel Coherence Score/Accuracy on Tweets: ', coherence_lda)


Perplexity:  -7.613607723703291

 Ldamodel Coherence Score/Accuracy on Tweets:  0.31468789739793684


The model perfomed poorly. The score of 0.31 is poor. we need to use maybe another model or finetune the hyperparameters or more of preprocessing to get the right data for this problem.

In [118]:
# ! pip install pyLDAvis 

     |████████████████████████████████| 1.7 MB 867 kB/s eta 0:00:01     |███████████████████▉            | 1.0 MB 744 kB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136882 sha256=f765b24be3028f94276d387448a555cf0eed232b15c8a8ae43a2b33de63db5f4
  Stored in directory: /Users/apple/Library/Caches/pip/wheels/57/a4/86/d10c6c2e0bf149fbc0afb0aa5a6528ac35b30a133a0270c477
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=cf5363e9b9d543efd291e96f24293fb4c2c8a2b824661186dee1de3bc14cb327
  Stored in directory: /Users/apple/Library/Caches/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built pyLDAvis sklearn


In [119]:
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
LDAvis_prepared

/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are depreca

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.217301 -0.001236       1        1  27.180309
9      0.206046  0.090039       2        1  22.839003
0      0.138561 -0.149580       3        1  11.554895
3      0.132390  0.104632       4        1  10.297131
4      0.013982  0.171243       5        1   8.549877
8     -0.123768 -0.328254       6        1   5.617362
7      0.084339 -0.106142       7        1   4.937832
1     -0.263145  0.038085       8        1   3.625249
6     -0.171294  0.111234       9        1   2.997689
2     -0.234411  0.069979      10        1   2.400653, topic_info=       Term          Freq         Total Category  logprob  loglift
103   https   9734.000000   9734.000000  Default  30.0000  30.0000
24    china  11424.000000  11424.000000  Default  29.0000  29.0000
100      co   9481.000000   9481.000000  Default  28.0000  28.0000
56        t  10633.000000  10633.000000  Default  27.0000  27.0000
116      on   6582.000000   6582.000000  Default  26.0000  26.0000
..      ...           ...           ...      ...      ...      ...
24    china     99.431407  11424.468258  Topic10  -4.8180  -1.0146
15       rt    100.191005  18423.621023  Topic10  -4.8104  -1.4849
227      up     66.180573    427.377580  Topic10  -5.2251   1.8641
58       to     74.069203  11538.676866  Topic10  -5.1125  -1.3190
280  russia     66.191560    899.651799  Topic10  -5.2249   1.1200

[467 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
650       3  0.975526        1
650       5  0.021984        1
753       5  0.995469       23
300       4  0.012453        3
300       6  0.986316        3
...     ...       ...      ...
5263      9  0.993032      yes
82        5  0.623358      you
82        9  0.376530      you
570       5  0.998879     your
1022     10  0.991064  youtube

[510 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 10, 1, 4, 5, 9, 8, 2, 7, 3])

In [120]:
data_africa= pd.read_json('/Users/apple/Desktop/10 academy/data/africa_twitter_data.json',lines=True)

In [17]:
data_africa.head()

,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,...,retweet_count,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status
0,2022-08-03 20:19:13+00:00,1554924874166149120,1554924874166149120,#Pelosi airplane landed safely in #Taiwan 🇹🇼 ...,False,"[0, 222]","{'hashtags': [{'text': 'Pelosi', 'indices': [0...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,0,False,False,en,NaN,NaN,NaN,NaN,NaN
1,2022-08-02 15:24:42+00:00,1554488368347680771,1554488368347680768,Watch the video of the beginning of the Chines...,False,"[0, 250]","{'hashtags': [{'text': 'Pelosi', 'indices': [1...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,...,0,3,False,False,en,"{'media': [{'id': 1554487694776012800, 'id_str...",0.0,NaN,NaN,NaN
2,2022-08-02 15:02:35+00:00,1554482802502127621,1554482802502127616,#Pelosi \n#Taipei \n#taiwan\n#XiJinping \n#Chi...,False,"[0, 71]","{'hashtags': [{'text': 'Pelosi', 'indices': [0...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,2,False,False,en,"{'media': [{'id': 1554482776140906497, 'id_str...",0.0,NaN,NaN,NaN
3,2022-08-01 13:51:42+00:00,1554102573635932160,1554102573635932160,#HOBIPALOOZA #LaAcademiaExpulsion #WEURO2022 #...,False,"[0, 146]","{'hashtags': [{'text': 'HOBIPALOOZA', 'indices...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,0,False,False,en,"{'media': [{'id': 1554102566010732544, 'id_str...",0.0,NaN,NaN,NaN
4,2022-07-31 20:02:20+00:00,1553833459440910338,1553833459440910336,#Pelosi\n#china\nChina Time ✌️ https://t.co/tE...,False,"[0, 28]","{'hashtags': [{'text': 'Pelosi', 'indices': [0...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,2,False,False,en,"{'media': [{'id': 1553833456853045256, 'id_str...",0.0,NaN,NaN,NaN


In [27]:
data_africa['lang']

0        en
1        en
2        en
3        en
4        en
         ..
28603    en
28604    en
28605    en
28606    en
28607    en
Name: lang, Length: 28608, dtype: object

In [28]:
df = data_africa[data_africa['lang']!='en']
df

,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,...,retweet_count,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status
